<a href="https://colab.research.google.com/github/paulyu8868/test/blob/main/%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score







In [8]:
def calculate_indicators(df):
    """기술적 지표 계산"""
    df = df.copy()

    # RSI 계산 (14일 기준)
    delta = df['Close'].diff()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)

    avg_gain = gains.rolling(window=14).mean()
    avg_loss = losses.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # 볼린저 밴드 (20일 기준)
    # 이동평균
    ma20 = df['Close'].rolling(window=20).mean()
    df['MA20'] = ma20

    # 표준편차
    std20 = df['Close'].rolling(window=20).std()

    # 상단/하단 밴드
    upper = ma20 + (2 * std20)
    lower = ma20 - (2 * std20)

    df['Upper'] = upper.values  # .values를 사용하여 numpy array로 변환
    df['Lower'] = lower.values

    # 밴드폭
    df['Width'] = ((upper - lower) / ma20 * 100).values

    # NaN 값 처리
    df = df.fillna(method='bfill').fillna(method='ffill')

    # 모든 컬럼의 소수점 처리
    for col in ['RSI', 'MA20', 'Upper', 'Lower', 'Width']:
        df[col] = df[col].map(lambda x: round_half_up_to_two(x) if not pd.isna(x) else x)

    return df

In [3]:
def get_data(ticker, start, end):
    """주가 데이터 불러오기"""
    df = yf.download(ticker, start=start, end=end)
    df = df.drop(columns=['Volume', 'Adj Close'])
    df.index = df.index.date

    # 호가 단위 0.01$ 적용
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].map(round_half_up_to_two)

    # 등락율 계산
    df['Return'] = df['Close'].pct_change()
    df['Return'] = df['Return'].map(lambda x: round_half_up_to_two(x*100))

    return df

def round_half_up_to_two(num):
    """소수점 셋째자리에서 반올림"""
    try:
        if isinstance(num, (float, int)):
            num_100 = num * 100
            if num_100 - int(num_100) >= 0.5:
                return (int(num_100) + 1) / 100
            else:
                return int(num_100) / 100
        else:
            return num
    except:
        return num

In [6]:
def infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period):
    """매매 시뮬레이션"""
    funds = initial_funds
    one_buy_amount = initial_funds / buy_portion
    holdings = 0
    buy_records = []
    trade_history = []
    trade_id = 1

    df_res = pd.DataFrame(index=range(start_idx, start_idx + simulation_period + 1),
                         columns=['날짜', '시가', '고가', '종가', '등락율',
                                'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                '보유 주식 수', '예수금', '총 평가액', '수익율(%)'])

    for i in range(start_idx, start_idx + simulation_period + 1):
        current_date = df.index[i]
        open_price = float(df['Open'].iloc[i].item())
        high_price = float(df['High'].iloc[i].item())
        close_price = float(df['Close'].iloc[i].item())
        return_val = float(df['Return'].iloc[i].item())
        prev_price = float(df['Close'].iloc[i-1].item()) if i > 0 else close_price

        df_res.at[i, '날짜'] = current_date
        df_res.at[i, '시가'] = open_price
        df_res.at[i, '고가'] = high_price
        df_res.at[i, '종가'] = close_price
        df_res.at[i, '등락율'] = f"{return_val}%"

        price = close_price

        df_res.at[i, 'LOC 매수'] = 0
        df_res.at[i, '수익 실현 매도'] = 0
        df_res.at[i, 'MOC 손절'] = 0

        # 매수 로직
        if holdings == 0 or price <= prev_price:
            qty = int(one_buy_amount / price)
            if funds >= qty * price:
                holdings += qty
                funds -= qty * price
                buy_records.append({
                    'id': trade_id,
                    'buy_date': current_date,
                    'buy_price': price,
                    'quantity': qty,
                    'days': 0,
                    'type': 'LOC 매수'
                })
                df_res.at[i, 'LOC 매수'] = qty
                trade_id += 1

        # 매도 로직
        if holdings > 0:
            new_buy_records = []
            total_sell = 0

            for record in buy_records:
                record['days'] += 1
                sell_price = record['buy_price'] * 1.1  # 10% 수익 목표

                if price >= sell_price:
                    funds += record['quantity'] * price
                    total_sell += record['quantity']
                    holdings -= record['quantity']
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round_half_up_to_two((price/record['buy_price'] - 1) * 100)
                    })
                elif record['days'] >= 30:
                    funds += record['quantity'] * price
                    df_res.at[i, 'MOC 손절'] = record['quantity']
                    holdings -= record['quantity']
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round_half_up_to_two((price/record['buy_price'] - 1) * 100)
                    })
                else:
                    new_buy_records.append(record)

            if total_sell > 0:
                df_res.at[i, '수익 실현 매도'] = total_sell

            buy_records = new_buy_records

        df_res.at[i, '보유 주식 수'] = holdings
        df_res.at[i, '예수금'] = round_half_up_to_two(funds)
        df_res.at[i, '총 평가액'] = round_half_up_to_two(funds + (price * holdings))
        df_res.at[i, '수익율(%)'] = ((funds + (price * holdings)) / initial_funds - 1) * 100

    final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period].item()))
    return round_half_up_to_two((final_value / initial_funds - 1) * 100), df_res, final_value, pd.DataFrame(trade_history)

def create_training_features(df, df_trades, target_return=1.1):
    """학습 데이터 생성"""
    features_list = []

    for _, trade in df_trades.iterrows():
        buy_date = pd.to_datetime(trade['매수일']).date()
        try:
            trade_data = df.loc[buy_date]

            features = {
                '회차': trade['회차'],
                'BB위치': (trade['매수가'] - float(trade_data['Lower'])) /
                       (float(trade_data['Upper']) - float(trade_data['Lower'])) * 100,
                'BB밴드폭': float(trade_data['Width']),
                'RSI': float(trade_data['RSI']),
                '과거30일수익률': df['Close'].loc[:buy_date].pct_change(30).iloc[-1] * 100,
                'MA20이격도': (trade['매수가'] - float(trade_data['MA20'])) /
                          float(trade_data['MA20']) * 100,
                '실제수익률': trade['수익률(%)'],
                'is_target_achieved': trade['수익률(%)'] >= (target_return - 1) * 100
            }

            features_list.append(features)

        except Exception as e:
            print(f"Error processing trade {trade['회차']}: {str(e)}")
            continue

    return pd.DataFrame(features_list)

def analyze_and_predict(df_features, feature_columns):
    """모델 학습 및 분석"""
    X = df_features[feature_columns]
    y = df_features['is_target_achieved']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    model.fit(X_train_scaled, y_train)

    importance = pd.DataFrame({
        '파라미터': feature_columns,
        '중요도(%)': model.feature_importances_ * 100
    }).sort_values('중요도(%)', ascending=False)

    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]

    print(f"\n=== 목표 수익률 10% 달성 분석 ===")

    print("\n[파라미터 중요도]")
    display(importance.style.format({'중요도(%)': '{:.2f}%'}))

    print("\n[모델 성능]")
    print(f"정확도: {accuracy_score(y_test, y_pred):.2%}")
    print(f"정밀도: {precision_score(y_test, y_pred):.2%}")
    print(f"재현율: {recall_score(y_test, y_pred):.2%}")

    print("\n[확률 구간별 실제 달성률]")
    prob_bins = np.arange(0, 1.1, 0.1)
    y_test_array = np.array(y_test)

    for i in range(len(prob_bins)-1):
        mask = (y_prob >= prob_bins[i]) & (y_prob < prob_bins[i+1])
        if mask.sum() > 0:
            actual_rate = y_test_array[mask].mean()
            count = mask.sum()
            print(f"예측확률 {prob_bins[i]*100:3.0f}~{prob_bins[i+1]*100:3.0f}%: "
                  f"실제달성률 {actual_rate:.2%} (거래수: {count})")

    return model, scaler


In [11]:
if __name__ == "__main__":
    # 1. 초기 설정
    start_date = '2011-01-01'
    end_date = '2024-12-31'
    initial_funds = 100000
    buy_portion = 10
    ticker = 'TQQQ'

    print("=== 백테스트 시작 ===")
    print(f"종목: {ticker}")
    print(f"기간: {start_date} ~ {end_date}")
    print(f"초기자금: ${initial_funds:,}")
    print(f"분할 횟수: {buy_portion}")

    try:
        # 2. 데이터 준비
        start_date_dt = datetime.strptime(start_date,'%Y-%m-%d')
        start_date_before_30 = (start_date_dt - timedelta(days=30)).strftime('%Y-%m-%d')
        end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
        next_day = end_date_dt + timedelta(days=1)
        end_day_next = next_day.strftime('%Y-%m-%d')

        print("\n데이터 준비 중...")
        # 데이터 불러오기 및 지표 계산
        df = get_data(ticker, start_date_before_30, end_day_next)
        df = calculate_indicators(df)

        # 실제 분석 시작일부터의 데이터 길이 계산
        actual_start_data = get_data(ticker, start_date, end_day_next)
        df_length = len(df) - len(actual_start_data)

        print(f"총 {len(df):,}개의 데이터 준비 완료")

        # 3. 매매 시뮬레이션 실행
        print("\n매매 시뮬레이션 실행 중...")
        df_res = pd.DataFrame()
        return_rate, df_res, final_value, df_trades = infinite_buy_simulation(
            df, df_res, initial_funds, buy_portion, df_length, len(df)-1-df_length)

        print("시뮬레이션 완료")
        print(f"총 거래 횟수: {len(df_trades):,}건")

        # 4. 학습 데이터 생성
        print("\n머신러닝 모델 학습 데이터 생성 중...")
        feature_columns = ['BB위치', 'BB밴드폭', 'RSI', '과거30일수익률', 'MA20이격도']
        df_features = create_training_features(df, df_trades, target_return=1.1)

        # 5. 모델 학습 및 분석
        print("모델 학습 및 분석 중...\n")
        model, scaler = analyze_and_predict(df_features, feature_columns)

        # 6. 결과 출력
        print("\n=== 백테스트 최종 결과 ===")
        print(f"초기자금: ${initial_funds:,}")
        print(f"최종자금: ${final_value:,.2f}")
        print(f"총 수익률: {return_rate:.2f}%")
        print(f"총 거래 횟수: {len(df_trades):,}건")
        print(f"목표 수익(10%) 달성 비율: {(df_trades['수익률(%)'] >= 10).mean():.2%}")

        # 7. 거래 내역 상세 분석
        print("\n=== 거래 내역 상세 분석 ===")

        # 수익/손실 거래 비율
        profit_trades = df_trades[df_trades['수익률(%)'] > 0]
        loss_trades = df_trades[df_trades['수익률(%)'] <= 0]

        print(f"\n수익 거래: {len(profit_trades):,}건 ({len(profit_trades)/len(df_trades):.2%})")
        print(f"손실 거래: {len(loss_trades):,}건 ({len(loss_trades)/len(df_trades):.2%})")

        # 평균 보유기간
        print(f"\n평균 보유기간: {df_trades['보유기간'].mean():.1f}일")
        print(f"최대 수익률: {df_trades['수익률(%)'].max():.2f}%")
        print(f"최대 손실률: {df_trades['수익률(%)'].min():.2f}%")

        # 월별 수익률 분석
        df_trades['매수일'] = pd.to_datetime(df_trades['매수일'])
        monthly_returns = df_trades.groupby(df_trades['매수일'].dt.strftime('%Y-%m'))['수익률(%)'].mean()

        print("\n=== 월별 평균 수익률 ===")
        display(monthly_returns.tail(12))

        # # 그래프로 수익률 분포 시각화
        # plt.figure(figsize=(12, 6))
        # plt.hist(df_trades['수익률(%)'], bins=50, alpha=0.7)
        # plt.title('거래별 수익률 분포')
        # plt.xlabel('수익률 (%)')
        # plt.ylabel('빈도')
        # plt.axvline(x=0, color='r', linestyle='--')
        # plt.show()

    except Exception as e:
        print(f"오류 발생: {str(e)}")

=== 백테스트 시작 ===
종목: TQQQ
기간: 2011-01-01 ~ 2024-12-31
초기자금: $100,000
분할 횟수: 10

데이터 준비 중...


[*********************100%***********************]  1 of 1 completed
<ipython-input-8-22677b742c97>:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill').fillna(method='ffill')
[*********************100%***********************]  1 of 1 completed


총 3,509개의 데이터 준비 완료

매매 시뮬레이션 실행 중...
시뮬레이션 완료
총 거래 횟수: 1,544건

머신러닝 모델 학습 데이터 생성 중...


<ipython-input-6-d15fee72100b>:118: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'BB위치': (trade['매수가'] - float(trade_data['Lower'])) /
<ipython-input-6-d15fee72100b>:119: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  (float(trade_data['Upper']) - float(trade_data['Lower'])) * 100,
<ipython-input-6-d15fee72100b>:120: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'BB밴드폭': float(trade_data['Width']),
<ipython-input-6-d15fee72100b>:121: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'RSI': float(trade_data['RSI']),
<ipython-input-6-d15fee72100b>:123: FutureWarning: Calling float on a single element Se

모델 학습 및 분석 중...



/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:745: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:745: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:745: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)



=== 목표 수익률 10% 달성 분석 ===

[파라미터 중요도]


,파라미터,중요도(%)
3,과거30일수익률,29.01%
1,BB밴드폭,21.24%
4,MA20이격도,20.69%
0,BB위치,16.52%
2,RSI,12.53%



[모델 성능]
정확도: 57.61%
정밀도: 62.10%
재현율: 73.91%

[확률 구간별 실제 달성률]
예측확률  30~ 40%: 실제달성률 100.00% (거래수: 2)
예측확률  40~ 50%: 실제달성률 52.27% (거래수: 88)
예측확률  50~ 60%: 실제달성률 50.82% (거래수: 61)
예측확률  60~ 70%: 실제달성률 46.55% (거래수: 58)
예측확률  70~ 80%: 실제달성률 77.14% (거래수: 70)
예측확률  80~ 90%: 실제달성률 78.57% (거래수: 28)
예측확률  90~100%: 실제달성률 100.00% (거래수: 2)

=== 백테스트 최종 결과 ===
초기자금: $100,000
최종자금: $701,860.71
총 수익률: 601.86%
총 거래 횟수: 1,544건
목표 수익(10%) 달성 비율: 60.88%

=== 거래 내역 상세 분석 ===

수익 거래: 1,127건 (72.99%)
손실 거래: 417건 (27.01%)

평균 보유기간: 19.5일
최대 수익률: 26.99%
최대 손실률: -63.62%

=== 월별 평균 수익률 ===


,수익률(%)
매수일,
2023-12,11.775000
2024-01,11.813750
2024-02,7.313000
2024-03,-8.989000
2024-04,9.496000
2024-05,11.990000
2024-06,10.777143
2024-07,-3.245000
2024-08,8.148889
